##### Import Libraries, Openmm must be intalled prior to this

In [1]:
from simtk.openmm.app import *
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import MDAnalysis as md
import xdrlib
import warnings
import time
from pdbfixer import PDBFixer
from sys import stdout
import numpy as np
from sklearn.preprocessing import StandardScaler

/mnt/bst/bdeng2/knasif/miniconda3/envs/MDSim/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Loading Raw PDB files

In [136]:
pdb0_file = 'data/villin_water.pdb'
pdb1_file = 'data/polyALA.pdb'
pdb2_file = 'data/polyGLY.pdb'
pdb3_file = 'data/polyGV.pdb'
pdb4_file = 'data/8xj3.pdb'

##### Fixing PDB files for missing residues and bonds

In [2]:
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
import os

def fix_pdb(pdb_id):
    path = os.getcwd()
    if len(pdb_id) != 4:
        print("Creating PDBFixer...")
        fixer = PDBFixer(filename=pdb_id)
        print("Finding missing residues...")
        fixer.findMissingResidues()

        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                print("ok")
                del fixer.missingResidues[key]

        print("Finding nonstandard residues...")
        fixer.findNonstandardResidues()
        print("Replacing nonstandard residues...")
        fixer.replaceNonstandardResidues()
        print("Removing heterogens...")
        fixer.removeHeterogens(keepWater=True)

        print("Finding missing atoms...")
        fixer.findMissingAtoms()
        print("Adding missing atoms...")
        fixer.addMissingAtoms()
        print("Adding missing hydrogens...")
        fixer.addMissingHydrogens(7.0)
        print("Writing PDB file...")

        fixed_pdb_file = os.path.join(path, "%s_fixed_pH_%s.pdb" % (pdb_id.split('.')[0], 7))
        with open(fixed_pdb_file, "w") as outfile:
            PDBFile.writeFile(
                fixer.topology, 
                fixer.positions, 
                outfile, 
                keepIds=True
            )
        return fixed_pdb_file

fixed_pdb = fix_pdb('data/7dmu.pdb')

if fixed_pdb:
    print(f"Fixed PDB saved at {fixed_pdb}")
else:
    print("PDB fixing failed.")


Creating PDBFixer...
Finding missing residues...
ok
ok
ok
ok
ok
ok
ok
Finding nonstandard residues...
Replacing nonstandard residues...
Removing heterogens...
Finding missing atoms...
Adding missing atoms...
Adding missing hydrogens...
Writing PDB file...
Fixed PDB saved at /mnt/bst/bdeng2/knasif/MD/data/7dmu_fixed_pH_7.pdb


##### MD Simulation and dataset creation, recommended to do in a separate .py file for larger simulation steps

In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np

# File paths
pdb5_file = 'data/7dmu_fixed_pH_7.pdb'
force_field_files = ['amber14-all.xml', 'amber14/tip3p.xml']

# Load initial coordinates from a PDB file
pdb = PDBFile(pdb5_file)

# Choosing force field parameters
ff = ForceField(*force_field_files)
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

# Experiment parameters
temperature = 300 * kelvin
friction_coeff = 1 / picosecond
time_step = 0.002 * picoseconds
total_steps = 100000
save_interval = 1000  # Interval at which to save the coordinates

# Calculate how many data points will be saved
num_data_points = total_steps // save_interval

# Integrator
integrator = LangevinIntegrator(temperature, friction_coeff, time_step)

# Set a seed for Langevin integrator for reproducibility
seed = 42
integrator.setRandomNumberSeed(seed)

# Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()

# Initialize an array to store the selected data points
positions_array = np.zeros((num_data_points, pdb.topology.getNumAtoms(), 3))

# Run the simulation and collect data at specified intervals
data_index = 0
for step in range(1, total_steps + 1):
    simulation.step(1)
    if step % save_interval == 0:
        state = simulation.context.getState(getPositions=True)
        positions_array[data_index] = state.getPositions(asNumpy=True).value_in_unit(nanometers)
        data_index += 1

# Initialize lists to store PDB information
residue_numbers = []
residue_names = []
atom_names = []

# Extract residue numbers, names, and atom names from the PDB file
with open(pdb5_file, 'r') as pdb_file:
    for line in pdb_file:
        if line.startswith('ATOM'):
            columns = line.split()
            atom_names.append(columns[2])
            residue_names.append(columns[3])
            residue_numbers.append(int(columns[5]))

# Determine the number of intervals (e.g., 1000th steps included)
num_intervals = total_steps // save_interval

# Define dtype for the structured array
dtype = [
    ('residue_number', 'i4'),
    ('residue_name', 'U3'),
    ('atom_name', 'U3'),
    ('x', 'f4'),
    ('y', 'f4'),
    ('z', 'f4')
]

structured_array = np.zeros((num_intervals, pdb.topology.getNumAtoms()), dtype=dtype)

# Fill the structured array with data from the selected positions and the PDB file
for interval_index in range(num_intervals):
    for atom_index in range(pdb.topology.getNumAtoms()):
        structured_array[interval_index, atom_index] = (
            residue_numbers[atom_index],
            residue_names[atom_index],
            atom_names[atom_index],
            positions_array[interval_index, atom_index, 0],
            positions_array[interval_index, atom_index, 1],
            positions_array[interval_index, atom_index, 2]
        )

# Save the structured array with selected positions and PDB information
np.save('data/7dmu_pos.npy', structured_array)

print("Dataset created")


Dataset created


##### Loading saved positions from MD simulations

In [3]:
positions=np.load('data/8sk7_pos_20ns.npy')

In [4]:
positions.shape

(1000, 26115)

##### Extracting Alpha-Carbons, Alpha Carbons are the backbone of the protein structure

In [6]:
ca_pos=positions[positions['atom_name'] == 'CA']

In [7]:
np.save('data/8sk7_ca_pos.npy',ca_pos)

In [2]:
ca_positions=np.load('data/8sk7_ca_pos.npy')

In [3]:
ca_positions.shape

(1653000,)

##### Reshaping Dataset in a 3D format (number of data points, number of residues, (x,y,z))

In [4]:
# last three columns, which correspond to the (x, y, z) coordinates
atomic_coordinates = np.stack((ca_positions['x'], ca_positions['y'], ca_positions['z']), axis=-1)

# Reshape the data for scaling: 
num_steps = 1000  # The number of data points in the dataset
num_res = len(ca_positions) // num_steps
atomic_coordinates = atomic_coordinates.reshape(num_steps,num_res, 3)

In [5]:
atomic_coordinates.shape

(1000, 1653, 3)

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


##### import necessary libraries for Machine Learning Model

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Layer, MultiHeadAttention, Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Reshape, Input, Add, Activation, BatchNormalization, add

##### Normalize and Reshape the dataset in a grid format to fit in CNN model 

In [40]:
# Normalize the data
scaler = StandardScaler()
positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

# Calculate grid size based on the number of residues
import math
grid_size = int(math.ceil(math.sqrt(num_res)))

# Reshape the data for Conv2D input
reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

for i in range(positions_normalized.shape[0]):
    for j in range(num_res):
        row = j // grid_size
        col = j % grid_size
        reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

# Create input (X) and output (y) data for the model
X = reshaped_data[:-1]
y = reshaped_data[1:]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##### Attention Layer Function

In [41]:
class SelfAttention(Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(SelfAttention, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)

    def call(self, inputs):
        # Attention mechanism where the query, key, and value are the same
        return self.attention(inputs, inputs)

##### MDNet Model

In [45]:
# Define Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# Register Mish as a custom activation function
tf.keras.utils.get_custom_objects().update({'mish': Activation(mish)})

In [46]:
def build_model(num_res):
    # Calculate grid size based on the number of atoms
    grid_size = int(math.ceil(math.sqrt(num_res)))

    # Build the Conv2D model with dynamic input shape
    input_layer = Input(shape=(grid_size, grid_size, 3))
    x = Conv2D(32, (3, 3), activation='mish', padding='same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    x = SelfAttention(embed_dim=32)(x)
    x = Conv2D(64, (3, 3), activation='mish', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(128, activation='mish')(x)
    x = Dense(grid_size * grid_size * 3)(x)
    output_layer = Reshape((grid_size, grid_size, 3))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model

In [47]:
model = build_model(num_res)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 41, 41, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 20, 20, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_2 (Bat  (None, 20, 20, 32)        128       
 chNormalization)                                                
                                                                 
 self_attention_1 (SelfAtte  (None, 20, 20, 32)        33568     
 ntion)                                                          
                                                           

In [48]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 3s 31ms/step - loss: 0.3556 - val_loss: 0.8122
Epoch 2/100
20/20 [==============================] - 0s 13ms/step - loss: 0.1125 - val_loss: 0.8052
Epoch 3/100
20/20 [==============================] - 0s 12ms/step - loss: 0.0791 - val_loss: 0.7995
Epoch 4/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0763 - val_loss: 0.7930
Epoch 5/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0568 - val_loss: 0.7912
Epoch 6/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0482 - val_loss: 0.7882
Epoch 7/100
20/20 [==============================] - 0s 12ms/step - loss: 0.0536 - val_loss: 0.7853
Epoch 8/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0481 - val_loss: 0.7818
Epoch 9/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0583 - val_loss: 0.7763
Epoch 10/100
20/20 [==============================] - 0s 13ms/step - loss: 0.0407 - val_loss: 0.7719

In [49]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 9ms/step - loss: 0.0218


0.021833930164575577

#### Denormalization

In [50]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 0s 3ms/step
Mean Absolute Error on Actual Data: 0.00414960981101557


#### d_i calculation

In [51]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))
    return distances

#### TM-score

In [52]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")

The TM-score for all atoms across all time steps is: 0.9999994634713573


##### Resnet50

In [57]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [58]:
def preprocess_data(atomic_coordinates, num_res):
    scaler = StandardScaler()
    positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

    grid_size = int(math.ceil(math.sqrt(num_res)))

    reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

    for i in range(positions_normalized.shape[0]):
        for j in range(num_res):
            row = j // grid_size
            col = j % grid_size
            reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

    X = reshaped_data[:-1]
    y = reshaped_data[1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, grid_size

In [59]:
# Build the ResNet50 model
def build_resnet50_model(grid_size):
    input_layer = Input(shape=(grid_size, grid_size, 3))
    base_model = ResNet50(include_top=False, weights=None, input_tensor=input_layer)
    x = Flatten()(base_model.output)
    x = Dense(128, activation='relu')(x)
    x = Dense(grid_size * grid_size * 3)(x)
    output_layer = Reshape((grid_size, grid_size, 3))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model

In [60]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [61]:
model = build_resnet50_model(grid_size)

In [62]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 41, 41, 3)]          0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 47, 47, 3)            0         ['input_6[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 21, 21, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 21, 21, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [63]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 28s 115ms/step - loss: 0.8194 - val_loss: 0.8072
Epoch 2/100
20/20 [==============================] - 1s 32ms/step - loss: 0.8003 - val_loss: 0.7900
Epoch 3/100
20/20 [==============================] - 1s 31ms/step - loss: 0.7832 - val_loss: 0.7730
Epoch 4/100
20/20 [==============================] - 1s 31ms/step - loss: 0.7663 - val_loss: 0.7563
Epoch 5/100
20/20 [==============================] - 1s 31ms/step - loss: 0.7498 - val_loss: 0.7399
Epoch 6/100
20/20 [==============================] - 1s 31ms/step - loss: 0.7335 - val_loss: 0.7237
Epoch 7/100
20/20 [==============================] - 1s 32ms/step - loss: 0.7175 - val_loss: 0.7078
Epoch 8/100
20/20 [==============================] - 1s 31ms/step - loss: 0.7017 - val_loss: 0.6921
Epoch 9/100
20/20 [==============================] - 1s 31ms/step - loss: 0.6862 - val_loss: 0.6767
Epoch 10/100
20/20 [==============================] - 1s 32ms/step - loss: 0.6710 - val_loss: 0.66

In [64]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

7/7 [==============================] - 1s 36ms/step - loss: 0.1326
Test loss: 0.1326153427362442


#### Denormalization

In [65]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 1s 4ms/step
Mean Absolute Error on Actual Data: 0.02372708994940061


#### d_i calculation

In [67]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))
    return distances

##### TM-Score

In [68]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")

The TM-score for all atoms across all time steps is: 0.9999779693213898


##### Alexnet

In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [32]:
def preprocess_data(atomic_coordinates, num_res):
    scaler = StandardScaler()
    positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

    grid_size = int(math.ceil(math.sqrt(num_res)))

    reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

    for i in range(positions_normalized.shape[0]):
        for j in range(num_res):
            row = j // grid_size
            col = j % grid_size
            reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

    X = reshaped_data[:-1]
    y = reshaped_data[1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, grid_size

In [33]:
# Build the AlexNet model
def build_alexnet_model(grid_size):
    input_layer = Input(shape=(grid_size, grid_size, 3))
    
    x = Conv2D(96, (3, 3), strides=1, activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), strides=2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=2)(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(384, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(384, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), strides=2)(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(grid_size * grid_size * 3)(x)  # Output layer with units to match reshaped data
    output_layer = Reshape((grid_size, grid_size, 3))(x)  # Reshape output to match target data shape

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model


In [34]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [35]:
model = build_alexnet_model(grid_size)

In [36]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 41, 41, 3)]       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 41, 41, 96)        2688      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 20, 20, 96)        0         
 g2D)                                                            
                                                                 
 batch_normalization_8 (Bat  (None, 20, 20, 96)        384       
 chNormalization)                                                
                                                                 
 conv2d_16 (Conv2D)          (None, 20, 20, 256)       221440    
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 10, 10, 256)       0   

In [37]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 3s 19ms/step - loss: 0.5364 - val_loss: 0.5175
Epoch 2/100
20/20 [==============================] - 0s 11ms/step - loss: 0.2491 - val_loss: 0.6010
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 0.1960 - val_loss: 0.4850
Epoch 4/100
20/20 [==============================] - 0s 11ms/step - loss: 0.1711 - val_loss: 0.4784
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 0.1646 - val_loss: 0.4598
Epoch 6/100
20/20 [==============================] - 0s 11ms/step - loss: 0.1652 - val_loss: 0.4712
Epoch 7/100
20/20 [==============================] - 0s 14ms/step - loss: 0.1775 - val_loss: 0.4918
Epoch 8/100
20/20 [==============================] - 0s 12ms/step - loss: 0.1781 - val_loss: 0.4317
Epoch 9/100
20/20 [==============================] - 0s 11ms/step - loss: 0.1501 - val_loss: 0.4413
Epoch 10/100
20/20 [==============================] - 0s 11ms/step - loss: 0.1446 - val_loss: 0.4109

In [38]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

7/7 [==============================] - 0s 18ms/step - loss: 0.0701
Test loss: 0.07012642920017242


##### Denormalization

In [39]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 0s 2ms/step
Mean Absolute Error on Actual Data: 0.012799374462352494


##### d_i Calculation

In [40]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))
    return distances

##### TM-Score

In [41]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")

The TM-score for all atoms across all time steps is: 0.9999943689849696


##### Lenet

In [42]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Reshape, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [43]:
def preprocess_data(atomic_coordinates, num_res):
    scaler = StandardScaler()
    positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

    grid_size = int(math.ceil(math.sqrt(num_res)))

    reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

    for i in range(positions_normalized.shape[0]):
        for j in range(num_res):
            row = j // grid_size
            col = j % grid_size
            reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

    X = reshaped_data[:-1]
    y = reshaped_data[1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, grid_size

In [44]:
# Build the LeNet model
def build_lenet_model(grid_size):
    input_layer = Input(shape=(grid_size, grid_size, 3))
    
    x = Conv2D(6, (5, 5), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(16, (5, 5), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dense(120, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(84, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(grid_size * grid_size * 3)(x)  # Output layer with units to match reshaped data
    output_layer = Reshape((grid_size, grid_size, 3))(x)  # Reshape output to match target data shape

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model

In [45]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [46]:
model = build_lenet_model(grid_size)

In [47]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 41, 41, 3)]       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 41, 41, 6)         456       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 20, 20, 6)         0         
 ng2D)                                                           
                                                                 
 batch_normalization_11 (Ba  (None, 20, 20, 6)         24        
 tchNormalization)                                               
                                                                 
 conv2d_21 (Conv2D)          (None, 20, 20, 16)        2416      
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 10, 10, 16)        0   

In [48]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 3s 20ms/step - loss: 0.5370 - val_loss: 0.6684
Epoch 2/100
20/20 [==============================] - 0s 10ms/step - loss: 0.3041 - val_loss: 0.6093
Epoch 3/100
20/20 [==============================] - 0s 10ms/step - loss: 0.2643 - val_loss: 0.5783
Epoch 4/100
20/20 [==============================] - 0s 10ms/step - loss: 0.2315 - val_loss: 0.5742
Epoch 5/100
20/20 [==============================] - 0s 9ms/step - loss: 0.2429 - val_loss: 0.4961
Epoch 6/100
20/20 [==============================] - 0s 10ms/step - loss: 0.2267 - val_loss: 0.4873
Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: 0.2143 - val_loss: 0.5109
Epoch 8/100
20/20 [==============================] - 0s 12ms/step - loss: 0.1910 - val_loss: 0.4768
Epoch 9/100
20/20 [==============================] - 0s 10ms/step - loss: 0.1949 - val_loss: 0.4497
Epoch 10/100
20/20 [==============================] - 0s 10ms/step - loss: 0.1986 - val_loss: 0.4162


In [49]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

7/7 [==============================] - 0s 6ms/step - loss: 0.0708
Test loss: 0.07083191722631454


##### Denormalization

In [50]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 0s 1ms/step
Mean Absolute Error on Actual Data: 0.01314761748861592


##### d_i Calculation

In [51]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))
    return distances

##### TM_Score

In [52]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")

The TM-score for all atoms across all time steps is: 0.9999940412543494


##### VGGNet

In [53]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Reshape, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [54]:
def preprocess_data(atomic_coordinates, num_res):
    scaler = StandardScaler()
    positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

    grid_size = int(math.ceil(math.sqrt(num_res)))

    reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

    for i in range(positions_normalized.shape[0]):
        for j in range(num_res):
            row = j // grid_size
            col = j % grid_size
            reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

    X = reshaped_data[:-1]
    y = reshaped_data[1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, grid_size


In [55]:
# Build the VGGNet model
def build_vggnet_model(grid_size):
    input_layer = Input(shape=(grid_size, grid_size, 3))
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(grid_size * grid_size * 3)(x)  # Output layer with units to match reshaped data
    output_layer = Reshape((grid_size, grid_size, 3))(x)  # Reshape output to match target data shape

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model

In [56]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [57]:
model = build_vggnet_model(grid_size)

In [58]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 41, 41, 3)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 41, 41, 64)        1792      
                                                                 
 conv2d_23 (Conv2D)          (None, 41, 41, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 20, 20, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_24 (Conv2D)          (None, 20, 20, 128)       73856     
                                                                 
 conv2d_25 (Conv2D)          (None, 20, 20, 128)       147584    
                                                           

In [59]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 5s 47ms/step - loss: 0.7909 - val_loss: 0.4212
Epoch 2/100
20/20 [==============================] - 0s 16ms/step - loss: 0.2305 - val_loss: 0.1567
Epoch 3/100
20/20 [==============================] - 0s 14ms/step - loss: 0.1615 - val_loss: 0.1389
Epoch 4/100
20/20 [==============================] - 0s 13ms/step - loss: 0.1554 - val_loss: 0.1382
Epoch 5/100
20/20 [==============================] - 0s 13ms/step - loss: 0.1539 - val_loss: 0.1362
Epoch 6/100
20/20 [==============================] - 0s 14ms/step - loss: 0.1519 - val_loss: 0.1365
Epoch 7/100
20/20 [==============================] - 0s 14ms/step - loss: 0.1504 - val_loss: 0.1359
Epoch 8/100
20/20 [==============================] - 0s 14ms/step - loss: 0.1499 - val_loss: 0.1371
Epoch 9/100
20/20 [==============================] - 0s 16ms/step - loss: 0.1492 - val_loss: 0.1350
Epoch 10/100
20/20 [==============================] - 0s 13ms/step - loss: 0.1477 - val_loss: 0.1362

In [60]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

7/7 [==============================] - 0s 33ms/step - loss: 0.1316
Test loss: 0.13160699605941772


##### Denormalization

In [61]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 0s 2ms/step
Mean Absolute Error on Actual Data: 0.02361227775241208


##### d_i Calculation

In [62]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))
    return distances

##### TM-Score

In [63]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")

The TM-score for all atoms across all time steps is: 0.9999781013109255


##### GoogleNet (Inception V1)

In [121]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, BatchNormalization, Concatenate, Dropout, Reshape
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [122]:
def preprocess_data(atomic_coordinates, num_res):
    scaler = StandardScaler()
    positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

    grid_size = int(math.ceil(math.sqrt(num_res)))

    reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

    for i in range(positions_normalized.shape[0]):
        for j in range(num_res):
            row = j // grid_size
            col = j % grid_size
            reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

    X = reshaped_data[:-1]
    y = reshaped_data[1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, grid_size


In [123]:
# Inception module
def inception_module(x, filters):
    f1, f3r, f3, f5r, f5, fp = filters

    conv1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)

    conv3 = Conv2D(f3r, (1, 1), padding='same', activation='relu')(x)
    conv3 = Conv2D(f3, (3, 3), padding='same', activation='relu')(conv3)

    conv5 = Conv2D(f5r, (1, 1), padding='same', activation='relu')(x)
    conv5 = Conv2D(f5, (5, 5), padding='same', activation='relu')(conv5)

    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = Conv2D(fp, (1, 1), padding='same', activation='relu')(pool)

    return Concatenate(axis=-1)([conv1, conv3, conv5, pool])

In [124]:
# Build the GoogleNet (Inception v1) model
def build_googlenet_model(grid_size):
    input_layer = Input(shape=(grid_size, grid_size, 3))

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [64, 96, 128, 16, 32, 32])
    x = inception_module(x, [128, 128, 192, 32, 96, 64])
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [192, 96, 208, 16, 48, 64])
    x = inception_module(x, [160, 112, 224, 24, 64, 64])
    x = inception_module(x, [128, 128, 256, 24, 64, 64])
    x = inception_module(x, [112, 144, 288, 32, 64, 64])
    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = inception_module(x, [384, 192, 384, 48, 128, 128])

    x = AveragePooling2D(pool_size=(grid_size // 16, grid_size // 16), strides=(1, 1))(x)  # Adjusted pooling size
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(grid_size * grid_size * 3)(x)  # Output layer with units to match reshaped data
    output_layer = Reshape((grid_size, grid_size, 3))(x)  # Reshape output to match target data shape

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model


In [125]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [126]:
model = build_googlenet_model(grid_size)

In [127]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [128]:
model = build_googlenet_model(grid_size)

In [129]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 41, 41, 3)]          0         []                            
                                                                                                  
 conv2d_263 (Conv2D)         (None, 21, 21, 64)           9472      ['input_13[0][0]']            
                                                                                                  
 max_pooling2d_71 (MaxPooli  (None, 11, 11, 64)           0         ['conv2d_263[0][0]']          
 ng2D)                                                                                            
                                                                                                  
 batch_normalization_21 (Ba  (None, 11, 11, 64)           256       ['max_pooling2d_71[0][0

In [130]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)


Epoch 1/100


2024-07-25 23:52:32.189669: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_10/dropout_16/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


20/20 [==============================] - 15s 45ms/step - loss: 0.4510 - val_loss: 0.6191
Epoch 2/100
20/20 [==============================] - 0s 24ms/step - loss: 0.1980 - val_loss: 0.5376
Epoch 3/100
20/20 [==============================] - 0s 24ms/step - loss: 0.1718 - val_loss: 0.4728
Epoch 4/100
20/20 [==============================] - 0s 23ms/step - loss: 0.1745 - val_loss: 0.4435
Epoch 5/100
20/20 [==============================] - 0s 22ms/step - loss: 0.1695 - val_loss: 0.3116
Epoch 6/100
20/20 [==============================] - 0s 22ms/step - loss: 0.1574 - val_loss: 0.2515
Epoch 7/100
20/20 [==============================] - 0s 24ms/step - loss: 0.1362 - val_loss: 0.2364
Epoch 8/100
20/20 [==============================] - 0s 23ms/step - loss: 0.1034 - val_loss: 0.1868
Epoch 9/100
20/20 [==============================] - 0s 24ms/step - loss: 0.0981 - val_loss: 0.1112
Epoch 10/100
20/20 [==============================] - 0s 24ms/step - loss: 0.0921 - val_loss: 0.0775
Epoch 11/1

In [131]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

7/7 [==============================] - 0s 5ms/step - loss: 0.0267
Test loss: 0.026706688106060028


##### Denormalization

In [132]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 0s 4ms/step
Mean Absolute Error on Actual Data: 0.005198861590368986


##### d_i Calculation

In [133]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))
    return distances

##### TM_Score

In [134]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")

The TM-score for all atoms across all time steps is: 0.9999991343262455


##### DenseNet

In [99]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, AveragePooling2D, GlobalAveragePooling2D, Dense, Reshape
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import math

In [100]:
def preprocess_data(atomic_coordinates, num_res):
    scaler = StandardScaler()
    positions_normalized = scaler.fit_transform(atomic_coordinates.reshape(-1, atomic_coordinates.shape[-1])).reshape(atomic_coordinates.shape)

    grid_size = int(math.ceil(math.sqrt(num_res)))

    reshaped_data = np.zeros((positions_normalized.shape[0], grid_size, grid_size, 3))  # Dynamically sized grid

    for i in range(positions_normalized.shape[0]):
        for j in range(num_res):
            row = j // grid_size
            col = j % grid_size
            reshaped_data[i, row, col, :] = positions_normalized[i, j, :]

    X = reshaped_data[:-1]
    y = reshaped_data[1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler, grid_size

In [101]:
# Dense Block
def dense_block(x, blocks, name):
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

In [102]:
# Convolution Block
def conv_block(x, growth_rate, name):
    x1 = BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_bn')(x)
    x1 = Activation('relu', name=name + '_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False, name=name + '_conv1')(x1)
    x1 = BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_bn2')(x1)
    x1 = Activation('relu', name=name + '_relu2')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_conv2')(x1)
    x = Concatenate(axis=3, name=name + '_concat')([x, x1])
    return x

In [103]:
# Transition Layer
def transition_block(x, reduction, name):
    x = BatchNormalization(axis=3, epsilon=1.001e-5, name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[3] * reduction), 1, use_bias=False, name=name + '_conv')(x)
    x = AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

In [104]:
# Build the DenseNet model
def build_densenet_model(grid_size):
    input_layer = Input(shape=(grid_size, grid_size, 3))

    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(input_layer)
    x = BatchNormalization(axis=3, epsilon=1.001e-5, name='conv1/bn')(x)
    x = Activation('relu', name='conv1/relu')(x)
    x = AveragePooling2D(3, strides=2, name='pool1')(x)

    x = dense_block(x, 6, name='conv2')
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, 12, name='conv3')
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, 24, name='conv4')
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, 16, name='conv5')

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(grid_size * grid_size * 3, activation='linear')(x)
    output_layer = Reshape((grid_size, grid_size, 3))(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mean_absolute_error')

    return model

In [105]:
X_train, X_test, y_train, y_test, scaler, grid_size = preprocess_data(atomic_coordinates, num_res)

In [106]:
model = build_densenet_model(grid_size)

In [107]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 41, 41, 3)]          0         []                            
                                                                                                  
 conv1/conv (Conv2D)         (None, 18, 18, 64)           9408      ['input_11[0][0]']            
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 18, 18, 64)           256       ['conv1/conv[0][0]']          
 on)                                                                                              
                                                                                                  
 conv1/relu (Activation)     (None, 18, 18, 64)           0         ['conv1/bn[0][0]']      

In [108]:
# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.2)

Epoch 1/100
20/20 [==============================] - 45s 122ms/step - loss: 0.2547 - val_loss: 0.7081
Epoch 2/100
20/20 [==============================] - 1s 63ms/step - loss: 0.1065 - val_loss: 0.5830
Epoch 3/100
20/20 [==============================] - 1s 60ms/step - loss: 0.0767 - val_loss: 0.4781
Epoch 4/100
20/20 [==============================] - 1s 58ms/step - loss: 0.0747 - val_loss: 0.3719
Epoch 5/100
20/20 [==============================] - 1s 58ms/step - loss: 0.0596 - val_loss: 0.3188
Epoch 6/100
20/20 [==============================] - 1s 57ms/step - loss: 0.0549 - val_loss: 0.2457
Epoch 7/100
20/20 [==============================] - 1s 62ms/step - loss: 0.0670 - val_loss: 0.2197
Epoch 8/100
20/20 [==============================] - 1s 59ms/step - loss: 0.0598 - val_loss: 0.1942
Epoch 9/100
20/20 [==============================] - 1s 58ms/step - loss: 0.0490 - val_loss: 0.1500
Epoch 10/100
20/20 [==============================] - 1s 58ms/step - loss: 0.0636 - val_loss: 0.14

In [109]:
# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}")

7/7 [==============================] - 1s 10ms/step - loss: 0.1795
Test loss: 0.17950527369976044


#### Denormalization

In [110]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# Grid dimensions based on the dataset
grid_size = int(math.ceil(np.sqrt(num_res)))  # Assuming num_residues has been calculated as the nearest perfect square

# Predict using the model on normalized test data
predicted_normalized = model.predict(X_test)

# Denormalize the predicted data
predicted = scaler.inverse_transform(predicted_normalized.reshape(-1, 3))
predicted = predicted.reshape(-1, grid_size, grid_size, 3)

# Denormalize the actual test data
actual = scaler.inverse_transform(y_test.reshape(-1, 3))
actual = actual.reshape(-1, grid_size, grid_size, 3)

# Calculate Mean Squared Error on the denormalized data
mae = mean_absolute_error(actual.reshape(-1, 3), predicted.reshape(-1, 3))
print("Mean Absolute Error on Actual Data:", mae)

7/7 [==============================] - 2s 8ms/step
Mean Absolute Error on Actual Data: 0.035060291518481036


##### d_i Calculation

In [111]:
import numpy as np

def calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size):
    # Determine the row and column in the grid for the atom index
    row = atom_index // grid_size
    col = atom_index % grid_size

    # Extract the coordinates for the specified atom over all time steps
    actual_atom_coords = actual[:, row, col, :]
    predicted_atom_coords = predicted[:, row, col, :]
    
    # Calculate and return the Euclidean distances for this atom over all time steps
    distances = np.sqrt(np.sum((actual_atom_coords - predicted_atom_coords) ** 2, axis=1))

##### TM-Score

In [114]:
# L_target is the total number of atoms in the protein structure.
L_target = num_res
L_common = num_res
grid_size = int(np.ceil(np.sqrt(L_target)))  # Calculate grid size dynamically

# Calculate d0, the normalization factor
d0 = 1.24 * np.cbrt(L_target - 15) - 1.8

# Initialize the TM-score sum
tm_score_sum = 0

# Iterate over all atoms to calculate the distance and contribute to the TM-score sum
for atom_index in range(L_common):
    distances = calculate_distance_for_one_atom_over_time(actual, predicted, atom_index, grid_size)
    tm_score_sum += np.sum(1 / (1 + (distances / d0) ** 2))

# Normalize the TM-score sum by the length of the target protein to get the final TM-score
tm_score = tm_score_sum / (L_target * actual.shape[0])

print(f"The TM-score for all atoms across all time steps is: {tm_score}")


TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'